In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
train_data = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')
test_data = pd.read_csv('../input/tmdb-box-office-prediction/test.csv')

In [ ]:
train_data.head()

In [ ]:
training_data = train_data.drop(['id', 'belongs_to_collection', 'homepage', 'imdb_id',
                                 'original_title', 'overview', 'poster_path', 'release_date', 'tagline', 'title',
                              'Keywords', 'cast', 'crew'
                             ], axis = 1)

In [ ]:
training_data.head()

In [ ]:
training_data.isnull().sum()

In [ ]:
training_data = training_data.fillna('0')

In [ ]:
training_data.isnull().sum()

In [ ]:
def process_json_data(series):
    str_list = []
    for i in series:
        string = []
        if (i != '0'):
            o = ast.literal_eval(i)
            for i in o:
                for j in i.items():
                    if (j[0] == 'name'):
                        string.append(j[1])
        str_list.append(' + '.join(string))
    return LabelEncoder().fit_transform(str_list)

In [ ]:
training_data.index = train_data['id']
training_data['genres'] = process_json_data(training_data['genres'])
training_data['production_companies'] = process_json_data(training_data['production_companies'])
training_data['production_countries'] = process_json_data(training_data['production_countries'])
training_data['spoken_languages'] = process_json_data(training_data['spoken_languages'])
training_data['original_language'] = LabelEncoder().fit_transform(training_data['original_language'])
training_data['status'] = LabelEncoder().fit_transform(training_data['status'])

In [ ]:
training_data.head()

In [ ]:
sns.heatmap(training_data.corr())

In [ ]:
y = training_data['revenue']
x = training_data.drop(['revenue'], axis=1)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [ ]:
testing_data = test_data.drop(['id', 'belongs_to_collection', 'homepage', 'imdb_id',
                                 'original_title', 'overview', 'poster_path', 'release_date', 'tagline', 'title',
                              'Keywords', 'cast', 'crew'
                             ], axis = 1)
testing_data = testing_data.fillna('0')
testing_data.index = test_data['id']
testing_data['genres'] = process_json_data(testing_data['genres'])
testing_data['production_companies'] = process_json_data(testing_data['production_companies'])
testing_data['production_countries'] = process_json_data(testing_data['production_countries'])
testing_data['spoken_languages'] = process_json_data(testing_data['spoken_languages'])
testing_data['original_language'] = LabelEncoder().fit_transform(testing_data['original_language'])
testing_data['status'] = LabelEncoder().fit_transform(testing_data['status'])

In [ ]:
results = model.predict(testing_data)

submission = pd.Dataframe({'id' : test_data['id'], 'revenue': results})
submission.to_csv('submission.csv', index=False)

# 3.48308